In [1]:
import pandapower as pp
import pandapower.networks as pn

net = pn.case33bw()


In [2]:
net.load.drop(net.load.index, inplace=True)


In [3]:
l1 = pp.create_load(net, bus=4,  p_mw=0.0, q_mvar=0.0)
l2 = pp.create_load(net, bus=11, p_mw=0.0, q_mvar=0.0)
l3 = pp.create_load(net, bus=17, p_mw=0.0, q_mvar=0.0)
l4 = pp.create_load(net, bus=24, p_mw=0.0, q_mvar=0.0)


In [4]:
pv1 = pp.create_sgen(net, bus=17, p_mw=0.0, q_mvar=0.0)
pv2 = pp.create_sgen(net, bus=24, p_mw=0.0, q_mvar=0.0)


In [5]:
battery = pp.create_sgen(net, bus=14, p_mw=0.0, q_mvar=0.0)


In [6]:
print(net.load)
print(net.sgen)


   name  bus  p_mw  q_mvar  const_z_p_percent  const_z_q_percent  \
0  None    4   0.0     0.0                0.0                0.0   
1  None   11   0.0     0.0                0.0                0.0   
2  None   17   0.0     0.0                0.0                0.0   
3  None   24   0.0     0.0                0.0                0.0   

   const_i_p_percent  const_i_q_percent  sn_mva  scaling  in_service type  \
0                0.0                0.0     NaN      1.0        True  wye   
1                0.0                0.0     NaN      1.0        True  wye   
2                0.0                0.0     NaN      1.0        True  wye   
3                0.0                0.0     NaN      1.0        True  wye   

   controllable  
0         False  
1         False  
2         False  
3         False  
   name  bus  p_mw  q_mvar  sn_mva  scaling  in_service type  current_source  \
0  None   17   0.0     0.0     NaN      1.0        True  wye            True   
1  None   24   0.0     

In [8]:
import numpy as np

hours = 24 * 60

def generate_load_profile(base_kw, hours):
    load = []
    for t in range(hours):
        hour = t % 24
        day = (t // 24) % 7

        daily = 0.6 + 0.4*np.sin((hour-7)/24 * 2*np.pi)**2
        weekly = 0.9 if day >= 5 else 1.0
        noise = np.random.normal(0, 0.03)

        value = base_kw * daily * weekly * (1 + noise)
        load.append(max(value, 0))

    return np.array(load)

load_bus5  = generate_load_profile(150, hours)
load_bus12 = generate_load_profile(120, hours)
load_bus18 = generate_load_profile(100, hours)
load_bus25 = generate_load_profile(90, hours)


In [9]:
def generate_pv_profile(max_kw, hours):
    pv = []
    for t in range(hours):
        hour = t % 24
        if 6 <= hour <= 18:
            irradiance = np.sin((hour-6)/12 * np.pi)
            cloud = np.random.uniform(0.8, 1.0)
            value = max_kw * irradiance * cloud
        else:
            value = 0
        pv.append(value)
    return np.array(pv)

pv_bus18 = generate_pv_profile(120, hours)
pv_bus25 = generate_pv_profile(100, hours)


In [10]:
import pandapower as pp
import pandapower.networks as pn

net = pn.case33bw()

net.load.drop(net.load.index, inplace=True)

l1 = pp.create_load(net, bus=4,  p_mw=0.0, q_mvar=0.0)
l2 = pp.create_load(net, bus=11, p_mw=0.0, q_mvar=0.0)
l3 = pp.create_load(net, bus=17, p_mw=0.0, q_mvar=0.0)
l4 = pp.create_load(net, bus=24, p_mw=0.0, q_mvar=0.0)

pv1 = pp.create_sgen(net, bus=17, p_mw=0.0, q_mvar=0.0)
pv2 = pp.create_sgen(net, bus=24, p_mw=0.0, q_mvar=0.0)
battery = pp.create_sgen(net, bus=14, p_mw=0.0, q_mvar=0.0)


In [11]:
battery_capacity = 300
soc = 0.5 * battery_capacity
max_batt_power = 80
eff = 0.95

for t in range(hours):

    net.load.at[l1, "p_mw"] = load_bus5[t] / 1000
    net.load.at[l2, "p_mw"] = load_bus12[t] / 1000
    net.load.at[l3, "p_mw"] = load_bus18[t] / 1000
    net.load.at[l4, "p_mw"] = load_bus25[t] / 1000

    total_load = load_bus5[t] + load_bus12[t] + load_bus18[t] + load_bus25[t]
    total_pv = pv_bus18[t] + pv_bus25[t]

    deficit = total_load - total_pv

    if deficit > 0 and soc > 0.2*battery_capacity:
        batt_power = min(deficit, max_batt_power)
        soc -= batt_power/eff
    elif deficit < 0 and soc < 0.95*battery_capacity:
        batt_power = -min(abs(deficit), max_batt_power)
        soc -= batt_power*eff
    else:
        batt_power = 0

    net.sgen.at[pv1, "p_mw"] = pv_bus18[t] / 1000
    net.sgen.at[pv2, "p_mw"] = pv_bus25[t] / 1000
    net.sgen.at[battery, "p_mw"] = batt_power / 1000

    pp.runpp(net)

    min_v = net.res_bus.vm_pu.min()
    # continue storing dataset...
